In [48]:
import pandas as pd

In [49]:
def get_state_name(lat, long):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat)+", "+str(long)
    location = locator.reverse(coordinates)
    return location.raw['address']["state"]

In [66]:
def get_value_per_acre(lat, long):
    propertydata = pd.read_csv("Data/statewise-pricing-usa.csv")
    state = get_state_name(lat, long)
    return propertydata.loc[propertydata['State'] == state]

In [67]:
get_value_per_acre(32.3182, -86.9023)

,State,"Land Value (Per Acre, As-Is)"
1,Alabama,105200


In [70]:
def get_lat_long_from_location(name):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(name)
    print((location.latitude,location.longitude))

In [71]:
get_lat_long_from_location('Cayuga')

(42.8093409, -76.5700777)
